# Spotify Music Popularity Capstone - Modeling

Greg Welliver   

In [1]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as plticker
import seaborn as sns
sns.set()
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats

In [2]:
# load the data
df = pd.read_parquet('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq.parquet')
df.head()

,Top100,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,Label_i! Records,Label_iDEAL,Label_independent,Label_mau5trap,Label_nterscope,"Label_will.i.am, A&M","Label_will.i.am, Cherrytree, Interscope, Dandyville",Label_~scape,Label_â„— 2019 Speakerbox Recordings Limited under,Label_å†¬æ¨¹ç¤¾
0,Y,0.030321,0.960526,0.057939,0.661,0.000000,0.0454,0.869679,0.059521,0.421389,...,0,0,0,0,0,0,0,0,0,0
1,Y,0.143574,0.853239,0.055064,0.806,0.000000,0.0771,0.843131,0.279917,0.382856,...,0,0,0,0,0,0,0,0,0,0
2,Y,0.037249,0.789474,0.083111,0.768,0.000002,0.5180,0.868416,0.247659,0.322836,...,0,0,0,0,0,0,0,0,0,0
3,Y,0.061847,0.860324,0.059530,0.700,0.000002,0.2440,0.834646,0.049740,0.410788,...,0,0,0,0,0,0,0,0,0,0
4,N,0.028213,0.733806,0.048665,0.640,0.000000,0.0990,0.812575,0.345473,0.669470,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#df = df.reset_index(drop=True)

In [4]:
df.shape

(255096, 24362)

### Split data into training and testing subsets

In [ ]:
# Create the X and y matrices from the dataframe
X = df.drop(columns = ['Top100'])
y = df['Top100']

In [ ]:
# from sklearn.model_selection import train_test_split; reserve 20% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=1)

#### write to CSV
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

#### write to parquet
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq.parquet')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_parquet(filepath, 
              engine = "pyarrow", 
              compression = None)

df = pd.read_parquet('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq.parquet')